In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
from tcn import TCN
import warnings
warnings.filterwarnings('ignore')


2024-02-28 11:32:48.024185: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-28 11:32:48.066793: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 11:32:48.066828: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-28 11:32:48.066849: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-28 11:32:48.074734: I tensorflow/core/platform/cpu_feature_g

In [2]:
# df['ML_Node'].value_counts()

In [3]:
selected_features = ['timestamp_seconds', # lowers the accuracy 
                     'node_memory_Percpu_bytes', 
                     'node_context_switches_total', 
                     'surfsara_power_usage', 
                     'node_netstat_Tcp_InSegs', 
                     'node_netstat_Tcp_OutSegs', 
                     'node_network_transmit_packets_total-sum', 
                     'node_filesystem_size_bytes-sum', 
                     'node_filesystem_files-sum', 
                     'node_memory_MemFree_bytes', 
                     'node_netstat_Tcp_InErrs']


In [9]:
file_path = "myData2.parquet"
df = pd.read_parquet(file_path)


In [10]:
# Extract relevant columns
df_selected = df[['timestamp', 'state'] + selected_features].copy()

# Encode the target variable 'state' to binary (0 for "COMPLETED", 1 otherwise)
df_selected['target'] = (df_selected['state'] != 'COMPLETED').astype(int)

# Drop the original 'state' column
df_selected.drop('state', axis=1, inplace=True)


In [5]:
# Define time intervals
time_intervals = {'minute': '1T', 'hour': '1H', 'day': '1D'}

# Normalize selected features
scaler = MinMaxScaler()
df_selected[selected_features] = scaler.fit_transform(df_selected[selected_features])

# Set sequence length
sequence_length = 30

# Number of time steps to predict into the future
prediction_steps_tcn = 7


In [6]:
# Function to prepare data for TCN
def prepare_tcn_data(data, time_interval):
    data.set_index('timestamp', inplace=True)
    data_resampled = data.resample(time_interval).sum()
    # data_resampled['target'] = data_resampled['target'].clip(upper=1)  # Clip values to 1
    return data_resampled

# Function to create sequences for TCN
def create_tcn_sequences(data, sequence_length):
    sequences, targets = [], []
    for i in range(len(data) - sequence_length):
        seq = data.iloc[i:i+sequence_length].values
        target = data.iloc[i+sequence_length]['target']
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

# Function to make predictions on new data for TCN model
def predict_future_failures_tcn(model, input_data, sequence_length, prediction_steps):
    predictions = []

    for _ in range(prediction_steps):
        # Make a prediction for the next time step
        prediction = model.predict(input_data.reshape(1, sequence_length, input_data.shape[1]))
        predictions.append(prediction[0, 0])

        # Shift the input data by one time step and append the new prediction
        input_data = np.roll(input_data, shift=-1, axis=0)
        input_data[-1, -1] = prediction[0, 0]

    return predictions


# Day

In [11]:
# Extract relevant columns
df_selected = df[['timestamp', 'state'] + selected_features].copy()

# Encode the target variable 'state' to binary (0 for "COMPLETED", 1 otherwise)
df_selected['target'] = (df_selected['state'] != 'COMPLETED').astype(int)

# Drop the original 'state' column
df_selected.drop('state', axis=1, inplace=True)


In [12]:
# Prepare data for TCN with day intervals
tcn_data_day = prepare_tcn_data(df_selected, time_intervals['day'])

# Create sequences and targets
sequences_day, targets_day = create_tcn_sequences(tcn_data_day, sequence_length)

# Split the data into training and testing sets
X_train_day, X_test_day, y_train_day, y_test_day = train_test_split(sequences_day, targets_day, test_size=0.3, random_state=1)

# Build the TCN model
tcn_model_day = Sequential([
    TCN(input_shape=(sequence_length, X_train_day.shape[2])),
    Dense(1, activation='sigmoid')
])

# Compile the model
tcn_model_day.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# Train the model
tcn_model_day.fit(X_train_day, y_train_day, epochs=20, batch_size=32, validation_split=0.15)


Epoch 1/20
2/2 [==============================] - 3s 514ms/step - loss: 6988846.5000 - mean_squared_error: 6988846.5000 - val_loss: 13315637.0000 - val_mean_squared_error: 13315637.0000
Epoch 2/20
2/2 [==============================] - 0s 81ms/step - loss: 6988846.5000 - mean_squared_error: 6988846.5000 - val_loss: 13315637.0000 - val_mean_squared_error: 13315637.0000
Epoch 3/20
2/2 [==============================] - 0s 82ms/step - loss: 6988846.5000 - mean_squared_error: 6988846.5000 - val_loss: 13315637.0000 - val_mean_squared_error: 13315637.0000
Epoch 4/20
2/2 [==============================] - 0s 86ms/step - loss: 6988846.5000 - mean_squared_error: 6988846.5000 - val_loss: 13315637.0000 - val_mean_squared_error: 13315637.0000
Epoch 5/20
2/2 [==============================] - 0s 80ms/step - loss: 6988846.5000 - mean_squared_error: 6988846.5000 - val_loss: 13315637.0000 - val_mean_squared_error: 13315637.0000
Epoch 6/20
2/2 [==============================] - 0s 84ms/step - loss: 698

In [ ]:
# Evaluate the model using Mean Squred Error
mae_day = tcn_model_day.evaluate(X_test_day, y_test_day, verbose=0)[1]
print(f'Model Mean Squared Error: {mae_day:.4f}\n')

# Select a starting point for predictions
input_data_tcn = X_test_day[3]

# Make predictions with the TCN model
predicted_failures_tcn = predict_future_failures_tcn(tcn_model_day, input_data_tcn, sequence_length, prediction_steps_tcn)

# Print the predicted failures for TCN
print("Predicted Failures for the Next 7 Time Steps (TCN):")
print(predicted_failures_tcn)

# Evaluate the predictions using Mean Squared Error
mse_predictions = np.mean((predicted_failures_tcn - y_test_day[3:3+prediction_steps_tcn])**2)
print(f'\nMean Squared Error for Predictions: {mse_predictions:.4f}\n')

# Evaluate the predictions using Mean Absolute Error
mae_predictions = np.mean(np.abs(predicted_failures_tcn - y_test_day[3:3+prediction_steps_tcn]))
print(f'Mean Absolute Error for Predictions: {mae_predictions:.4f}\n')


Model Mean Squared Error: 0.1923

1/1 [==============================] - 0s 23ms/step
Predicted Failures for the Next 7 Time Steps (TCN):
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Mean Squared Error for Predictions: 0.0000
Mean Absolute Error for Predictions: 0.0000



# Hour

In [ ]:
# Prepare data for TCN
tcn_data_hour = prepare_tcn_data(df_selected, time_intervals['hour'])

# Create sequences and targets
sequences_hour, targets_hour = create_tcn_sequences(tcn_data_hour, sequence_length)

# Split the data into training and testing sets
X_train_hour, X_test_hour, y_train_hour, y_test_hour = train_test_split(sequences_hour, targets_hour, test_size=0.3, random_state=1)

# Build the TCN model
tcn_model_hour = Sequential([
    TCN(input_shape=(sequence_length, X_train_hour.shape[2])),
    Dense(1, activation='sigmoid')
])

# Compile the model
tcn_model_hour.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# Train the model
tcn_model_hour.fit(X_train_hour, y_train_hour, epochs=20, batch_size=32, validation_split=0.15)


Epoch 1/20
51/51 [==============================] - 6s 53ms/step - loss: 0.4542 - mean_squared_error: 0.4542 - val_loss: 0.4129 - val_mean_squared_error: 0.4129
Epoch 2/20
51/51 [==============================] - 2s 46ms/step - loss: 0.4542 - mean_squared_error: 0.4542 - val_loss: 0.4128 - val_mean_squared_error: 0.4128
Epoch 3/20
51/51 [==============================] - 2s 45ms/step - loss: 0.4542 - mean_squared_error: 0.4542 - val_loss: 0.4128 - val_mean_squared_error: 0.4128
Epoch 4/20
51/51 [==============================] - 2s 44ms/step - loss: 0.4542 - mean_squared_error: 0.4542 - val_loss: 0.4128 - val_mean_squared_error: 0.4128
Epoch 5/20
51/51 [==============================] - 2s 44ms/step - loss: 0.4541 - mean_squared_error: 0.4541 - val_loss: 0.4127 - val_mean_squared_error: 0.4127
Epoch 6/20
51/51 [==============================] - 2s 43ms/step - loss: 0.4541 - mean_squared_error: 0.4541 - val_loss: 0.4127 - val_mean_squared_error: 0.4127
Epoch 7/20
51/51 [================

In [ ]:
# Evaluate the model using Mean Squred Error
mae_hour = tcn_model_hour.evaluate(X_test_hour, y_test_hour, verbose=0)[1]
print(f'Model Mean Squared Error: {mae_hour:.4f}\n')

# Select a starting point for predictions
input_data_tcn = X_test_hour[3]

# Make predictions with the TCN model
predicted_failures_tcn = predict_future_failures_tcn(tcn_model_hour, input_data_tcn, sequence_length, prediction_steps_tcn)

# Print the predicted failures for TCN
print("Predicted Failures for the Next 7 Time Steps (TCN):")
print(predicted_failures_tcn)

# Evaluate the predictions using Mean Squared Error
mse_predictions = np.mean((predicted_failures_tcn - y_test_hour[3:3+prediction_steps_tcn])**2)
print(f'\nMean Squared Error for Predictions: {mse_predictions:.4f}\n')

# Evaluate the predictions using Mean Absolute Error
mae_predictions = np.mean(np.abs(predicted_failures_tcn - y_test_hour[3:3+prediction_steps_tcn]))
print(f'Mean Absolute Error for Predictions: {mae_predictions:.4f}\n')


Model Mean Squared Error: 0.4503

1/1 [==============================] - 0s 24ms/step
Predicted Failures for the Next 7 Time Steps (TCN):
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Mean Squared Error for Predictions: 0.2857

Mean Absolute Error for Predictions: 0.2857



# Minute

In [ ]:
# Prepare data for TCN
tcn_data_minute = prepare_tcn_data(df_selected, time_intervals['minute'])

# Create sequences and targets
sequences_minute, targets_minute = create_tcn_sequences(tcn_data_minute, sequence_length)

# Split the data into training and testing sets
X_train_minute, X_test_minute, y_train_minute, y_test_minute = train_test_split(sequences_minute, targets_minute, test_size=0.3, random_state=1)

# Build the TCN model
tcn_model_minute = Sequential([
    TCN(input_shape=(sequence_length, X_train_minute.shape[2])),
    Dense(1, activation='sigmoid')
])

# Compile the model
tcn_model_minute.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# Train the model
tcn_model_minute.fit(X_train_minute, y_train_minute, epochs=10, batch_size=64, validation_split=0.15)


Epoch 1/10
1539/1539 [==============================] - 96s 61ms/step - loss: 0.4057 - mean_squared_error: 0.4057 - val_loss: 0.3906 - val_mean_squared_error: 0.3906
Epoch 2/10
1539/1539 [==============================] - 92s 60ms/step - loss: 0.3882 - mean_squared_error: 0.3882 - val_loss: 0.3834 - val_mean_squared_error: 0.3834
Epoch 3/10
1539/1539 [==============================] - 92s 60ms/step - loss: 0.3844 - mean_squared_error: 0.3844 - val_loss: 0.3814 - val_mean_squared_error: 0.3814
Epoch 4/10
1539/1539 [==============================] - 93s 60ms/step - loss: 0.3833 - mean_squared_error: 0.3833 - val_loss: 0.3807 - val_mean_squared_error: 0.3807
Epoch 5/10
1539/1539 [==============================] - 93s 60ms/step - loss: 0.3828 - mean_squared_error: 0.3828 - val_loss: 0.3805 - val_mean_squared_error: 0.3805
Epoch 6/10
1539/1539 [==============================] - 93s 61ms/step - loss: 0.3826 - mean_squared_error: 0.3826 - val_loss: 0.3803 - val_mean_squared_error: 0.3803
Epoc

In [ ]:
# Evaluate the model using Mean Squred Error
mae_minute = tcn_model_minute.evaluate(X_test_minute, y_test_minute, verbose=0)[1]
print(f'Model Mean Squared Error: {mae_hour:.4f}\n')

# Select a starting point for predictions
input_data_tcn = X_test_minute[3]

# Make predictions with the TCN model
predicted_failures_tcn = predict_future_failures_tcn(tcn_model_minute, input_data_tcn, sequence_length, prediction_steps_tcn)

# Print the predicted failures for TCN
print("Predicted Failures for the Next 7 Time Steps (TCN):")
print(predicted_failures_tcn)

# Evaluate the predictions using Mean Squared Error
mse_predictions = np.mean((predicted_failures_tcn - y_test_minute[3:3+prediction_steps_tcn])**2)
print(f'\nMean Squared Error for Predictions: {mse_predictions:.4f}\n')

# Evaluate the predictions using Mean Absolute Error
mae_predictions = np.mean(np.abs(predicted_failures_tcn - y_test_minute[3:3+prediction_steps_tcn]))
print(f'Mean Absolute Error for Predictions: {mae_predictions:.4f}\n')


Model Mean Squared Error: 0.4503

1/1 [==============================] - 0s 24ms/step
Predicted Failures for the Next 7 Time Steps (TCN):
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

Mean Squared Error for Predictions: 0.5714

Mean Absolute Error for Predictions: 0.5714

